In [ ]:
import sys
!{sys.executable} -m pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
# new import statements
from typing import Tuple, List
from tqdm import tqdm

### Retailrocket recommender system dataset

Source: https://www.kaggle.com/retailrocket/ecommerce-dataset

Load the dataset files.

In [ ]:
events = pd.read_csv(os.path.join(\
    '..', 'timeseries_lec_data', 'events.csv'))
item_properties = pd.read_csv(os.path.join(\
    '..', 'timeseries_lec_data', 'item_properties_part1.csv'))
category_tree = pd.read_csv(os.path.join(\
    '..', 'timeseries_lec_data', 'category_tree.csv'))

In [ ]:
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit='ms')
events['event_date'] = events['event_datetime'].dt.date
events

In [ ]:
grouped_events = events.groupby(['event_date', 'itemid', 'event']).size().\
                                reset_index(name='event_count')
grouped_events

In [ ]:
item_properties

In [ ]:
category_tree

#### Q1: What is the Spearman correlation between the number of "view" events and the number of "addtocart" events per item?

In [ ]:
spearman_corr = merged_df['view_count'].corr(merged_df['addtocart_count'],\
                                             method='spearman')
spearman_corr

#### Q2: Create a scatter plot with ordinary least squares' trend line to show correlation between the number of "view" events and the number of "addtocart" events per item. 

In [ ]:
fig = px.scatter(merged_df, x="view_count", y="addtocart_count", \
                 trendline="ols", trendline_color_override="red")

fig.write_html('scatter_plot_with_ols_trendline.html')
fig.show()

#### Q3: Plot a line chart of the number of events (view, transaction, etc.) over time, color-coded by the event type.

In [ ]:
events_grouped = events.groupby(['event_date', 'event']).size().reset_index(name='event_count')

fig = px.line(events_grouped, x='event_date', y='event_count', 
              color='event', 
              title="Event counts over time (by event type)", 
              labels={'event_count': 'Number of events', 'event_date': 'Date'})
fig.write_html('events_vs_date.html')
fig.show()

#### pandas `resample` method

Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

#### Q4: Calculate the monthly count number of events and create a line chart.

In [ ]:
# Set timestamp as the index, resample to Month-Start, and calculate the count of number of events
monthly_avg = events.set_index('event_datetime').resample('MS').size().reset_index(name='event_count')

monthly_avg['year'] = monthly_avg['event_datetime'].dt.year

fig = px.line(
    monthly_avg, 
    x='event_datetime', 
    y='event_count', 
    color='year', 
    title="Monthly total event counts",
    labels={'event_datetime': 'Month', 'event_count': 'Average Events', 'year': 'Year'}
)
fig.show()

#### Q5: Calculate the total number of events per day of the week for each year and visualize the trend with a line chart.

In [ ]:
events_v2 = events.copy()

events_v2['year'] = events_v2['event_datetime'].dt.year
events_v2['dayofweek'] = events_v2['event_datetime'].dt.dayofweek + 1  # Add 1 to make Monday = 1, Sunday = 7

events_v2 = events_v2.groupby(['year', 'dayofweek']).size().\
reset_index(name='event_count')

fig = px.line(
    events_v2, 
    x="dayofweek", 
    y="event_count", 
    color="year", 
    title="Total events per day of the week",
    labels={'dayofweek': 'Day of Week (1=Monday, 7=Sunday)', 'event_count': 'Total Events'}
)
fig.show()

In [ ]:
events

#### Q6: Aggregate the number of events per day and plot the trend for the total number of events over time.

In [ ]:
events_per_day = events.set_index('event_datetime').resample('D').\
size().reset_index(name='event_count')

fig = px.line(
    events_per_day, 
    x='event_datetime', 
    y='event_count', 
    title="Total events per day", 
    labels={'event_datetime': 'Date', 'event_count': 'Total Events'}
)
fig.show()

### One-hot encoding

One-hot encoding is a method for converting categorical data (non-numeric data that can take on a limited number of values, e.g., "view", "addtocart", "transaction") into a numerical format that machine learning algorithms can understand

#### `pandas get_dummies` function

Documentation: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

#### Q7: One-hot encode the events data

Let's first determine the type of `event` column.

In [ ]:
print(events['event'].dtype)

In [ ]:
def one_hot_encoder(df: pd.DataFrame, nan_as_category: bool = True) -> Tuple[pd.DataFrame, List[str]]:
    """
    One-hot encode categorical columns in the given DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        nan_as_category (bool): Whether to include NaN as a separate category.
    
    Returns:
        Tuple[pd.DataFrame, List[str]]: The transformed DataFrame and the list of new column names.
    """
    original_columns = list(df.columns)
    print(original_columns)
    
    categorical_columns = df.select_dtypes(["category", "object"]).columns.tolist()
    print(categorical_columns)
    
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    
    new_columns = [col for col in df.columns if col not in original_columns]
    print(new_columns)
        
    return df, new_columns

In [ ]:
events

In [ ]:
events_encoded, event_columns = one_hot_encoder(events)
events_encoded

## Statistical model for timeseries analysis

### 1. Exponential Moving Average (EMA)
EMA is a quantitative technique used as a forecasting model for time series analysis. It is a statistical method used to smooth time-series data by giving more weight to recent observations and less weight to older ones. It's useful in scenarios where recent data points are considered more relevant and informative. Typical use case scenarios include financial time-series analysis or e-commerce product popularity prediction.

#### Q8: Use an Exponential Moving Average (EMA) model to calculate the average itemid view counts per category in the events dataset. Calculate the EMA for each category over a specified span.

In [ ]:
item_properties

#### Filter `item_properties` to get only rows where `property` is 'categoryid'.

In [ ]:
category_items = item_properties[item_properties['property'] == 'categoryid'][['itemid', 'value']]
# 'value' column contains the category IDs, so let's rename value to categoryid
category_items.columns = ['itemid', 'categoryid']  
category_items

#### Type conversion to `int`.

In [ ]:
category_items['categoryid'] = category_items['categoryid'].astype(int)
category_tree['categoryid'] = category_tree['categoryid'].astype(int)

#### Merge `category_items` with `category_tree` to get `parentid` (product family) for each category.

In [ ]:
category_items = pd.merge(category_items, category_tree, on='categoryid', how='left')
category_items

#### Compute the unique category ids (product families).

In [ ]:
categories = category_items['categoryid'].unique()
categories

#### Create a dataframe to store the results

In [ ]:
results = pd.DataFrame(columns=["timestamp", "categoryid", "ema"])
results

#### Compute EMA for view event counts

In [ ]:
results_list = []

for category in tqdm(categories, total=len(categories)):
    # get the itemids associated with the current category
    category_items_for_category = category_items[category_items['categoryid'] == category]['itemid']
    
    # filter the events dataset for the current category's items and 'view' events
    category_events = events[(events['itemid'].isin(category_items_for_category)) & (events['event'] == 'view')]
    
    # group by timestamp to get the view counts per timestamp
    view_counts = category_events.groupby("timestamp").size()
    
    # apply the Exponential Moving Average (EMA) with a span of 16 to smooth the view counts
    ema_values = view_counts.ewm(span=16, adjust=False).mean()
    
    # store the results with the EMA values for the category
    category_results = pd.DataFrame({
        "timestamp": ema_values.index,
        "categoryid": category,
        "ema": ema_values.values
    })
    
    # append the current category's results to the results list
    results_list.append(category_results)

# concatenate all the results into a single DataFrame
results = pd.concat(results_list, ignore_index=True)

In [ ]:
results